In [1]:
import sys
sys.path.append('../../../')
from utils.packages import *
from utils.ml_fairness import *
from utils.standard_data import *
dir = 'res/titanic/'
Path(dir).mkdir(parents=True, exist_ok=True)

d_fields = ['Name', 'Stage', 'CVR', 'CVD', 'V_SPD', 'V_EOD', 'V_AOD', 'V_ERD', 'Acc', 'F1','SPD', 'EOD', 'AOD', 'ERD']
diff_file = dir + 'diff' + '.csv'
if(not os.path.isfile(diff_file)):
    with open(diff_file, 'a') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(d_fields)
    
f_count = len([name for name in os.listdir(dir) if os.path.isfile(os.path.join(dir, name)) and not name.startswith('.')])
fields = ['Acc', 'F1', 'DI','SPD', 'EOD', 'AOD', 'ERD', 'CNT', 'TI']
filename = dir + str(f_count) + '.csv'
with open(filename, 'a') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)

In [2]:
# Load data
train = pd.read_csv('../../../data/titanic/train.csv')
test = pd.read_csv('../../../data/titanic/test.csv')
df = train

In [3]:
## BASIC PREP
df['Sex'] = df['Sex'].replace({'female': 0.0, 'male': 1.0})

y1_df = df.copy()
## Custom feature
df["Age"] = df["Age"].fillna(-0.5)
bins = [-1, 0, 5, 12, 18, 24, 35, 60, np.inf]
labels = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Senior']
df['AgeGroup'] = pd.cut(df["Age"], bins, labels = labels)

df["CabinBool"] = (df["Cabin"].notnull().astype('int'))

df['Title'] = df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
df['Title'] = df['Title'].replace(['Lady', 'Capt', 'Col',
    'Don', 'Dr', 'Major', 'Rev', 'Jonkheer', 'Dona'], 'Rare')
    
df['Title'] = df['Title'].replace(['Countess', 'Lady', 'Sir'], 'Royal')
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Royal": 5, "Rare": 6}
df['Title'] = df['Title'].map(title_mapping)
df['Title'] = df['Title'].fillna(0)

age_title_mapping = {1: "Young Adult", 2: "Student", 3: "Adult", 4: "Baby", 5: "Adult", 6: "Adult"}
for x in range(len(train["AgeGroup"])):
    if df["AgeGroup"][x] == "Unknown":
        df["AgeGroup"][x] = age_title_mapping[train["Title"][x]]

age_mapping = {'Baby': 1, 'Child': 2, 'Teenager': 3, 'Student': 4, 'Young Adult': 5, 'Adult': 6, 'Senior': 7}
df['AgeGroup'] = df['AgeGroup'].map(age_mapping)

df = df.fillna({"Embarked": "S"})
embarked_mapping = {"S": 1, "C": 2, "Q": 3}
df['Embarked'] = df['Embarked'].map(embarked_mapping)
df['FareBand'] = pd.qcut(df['Fare'], 4, labels = [1, 2, 3, 4])


df = df.drop(['Fare'], axis = 1)
df = df.drop(['Cabin'], axis = 1)
df = df.drop(['Ticket'], axis = 1)
df = df.drop(['PassengerId'], axis = 1)
df = df.drop(['Name'], axis = 1)


y1_df['Age'].fillna(y1_df['Age'].median(), inplace = True)
y1_df = y1_df.fillna({"Embarked": "S"})
y1_df['Embarked'] = y1_df['Embarked'].map(embarked_mapping)
y1_df['Cabin'].fillna(y1_df['Cabin'].mode(), inplace = True)

# One-hot encoder
cat_feat = ['Cabin', 'Ticket', 'Embarked']
y1_df = pd.get_dummies(y1_df, columns=cat_feat, prefix_sep='=')


y1_df = y1_df.drop(['PassengerId'], axis = 1)
y1_df = y1_df.drop(['Name'], axis = 1)

In [4]:
for i in range(5):
    seed = randrange(100)
    y2_train, y2_test = train_test_split(df, test_size = 0.3, random_state = seed) # stratify=df['loan']
    y1_train, y1_test = train_test_split(y1_df, test_size = 0.3, random_state = seed) # 

    pro_att_name = ['Sex']
    priv_class = [1]
    reamining_cat_feat = []

    y2_data_orig_train, y2_X_train, y2_y_train = load_titanic_data(y2_train, pro_att_name, priv_class, reamining_cat_feat)
    y2_data_orig_test, y2_X_test, y2_y_test = load_titanic_data(y2_test, pro_att_name, priv_class, reamining_cat_feat)

    y1_data_orig_train, y1_X_train, y1_y_train = load_titanic_data(y1_train, pro_att_name, priv_class, reamining_cat_feat)
    y1_data_orig_test, y1_X_test, y1_y_test = load_titanic_data(y1_test, pro_att_name, priv_class, reamining_cat_feat)




    from sklearn.ensemble import GradientBoostingClassifier
    y2_model = GradientBoostingClassifier()
    y2_mdl = y2_model.fit(y2_X_train,y2_y_train) 
    
    y1_model = GradientBoostingClassifier()
    y1_mdl = y1_model.fit(y1_X_train,y1_y_train) 


    # plot_model_performance(y2_mdl, y2_X_test, y2_y_test)
    y1_pred, y1_fair = get_fair_metrics_and_plot(filename, y1_data_orig_test, y1_mdl)
    y2_pred, y2_fair = get_fair_metrics_and_plot(filename, y2_data_orig_test, y2_mdl)



    y1_fair = y1_fair.drop(['DI', 'CNT', 'TI'], axis=1)
    y2_fair = y2_fair.drop(['DI', 'CNT', 'TI'], axis=1)
    CVR, CVD, AVR_EOD, AVD_EOD, AVR_SPD, AVD_SPD, AVD_AOD, AV_ERD = compute_new_metrics(y2_data_orig_test, y1_pred, y2_pred)
    row_y1 = y1_fair.iloc[[0]].values[0].tolist()
    row_y2 = y2_fair.iloc[[0]].values[0].tolist()
    diff = []

    diff.append(CVR)
    diff.append(CVD)
    diff.append(AVD_SPD)
    diff.append(AVD_EOD)
    diff.append(AVD_AOD)
    diff.append(AV_ERD)

    for i in range(len(row_y2)):
        if(i < 2):
            change = row_y2[i] - row_y1[i]
        else:
            sign = ''
            if(row_y2[i] >= 0 and row_y1[i] >= 0):
                sign = '(+)'
                d = abs(row_y2[i]) - abs(row_y1[i])
            if(row_y2[i] < 0 and row_y1[i] < 0):
                sign = '(-)'
                d = abs(row_y2[i]) - abs(row_y1[i])
            if(row_y2[i] < 0 and row_y1[i] >= 0):
                sign = '(+-)'
                d = row_y2[i] - row_y1[i]
            if(row_y2[i] >=0 and row_y1[i] < 0):
                sign = '(-+)'
                d = row_y2[i] - row_y1[i]
            d = round(d, 3)
            change = sign + ' ' + str(d)

        diff.append(change)

    cols = ['CVR', 'CVD ', 'AV_SPD', 'AV_EOD', 'AV_AOD', 'AV_ERD', 'Acc', 'F1','SPD', 'EOD', 'AOD', 'ERD']
    # metrics = pd.DataFrame(data=obj_fairness, index=['y1'], columns=cols)
    diff_df = pd.DataFrame(data=[diff], columns  = cols, index = ['Diff']).round(3)
    stage = 'Custom(feature)'
    model_name = 'titanic5'
    diff = diff_df.iloc[0].values.tolist()
    diff.insert(0, stage)
    diff.insert(0, model_name)
    with open(diff_file, 'a') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(diff)    
        diff_df

Unprinv: Sex 0.0
Unprinv: Sex 0.0
Unprinv: Sex 0.0
Unprinv: Sex 0.0
Unprinv: Sex 0.0
